# **INDIEZ PERFORMANCE TEST**

## Position: **Game LiveOps Analyst**

## <span style="color: var(--vscode-foreground);">Name: <b>Vo Tu Anh</b></span>

# **Scenario**

INDIEZ is a gaming company who aims to bring the best experience to their users through hypercasual game products.

One of their games, "Pixel Art", has been under reconfiguration from version 1.5.2 to 1.6.0 with renovation in the in-game tutorial. The LiveOps analysts of the company believe that changes in the tutorial will result in better user experience (UX), hence enhancing the roll-out rate of the game.

# **Defining the problem**

The main problem is to **_improve the user experience (UX) in the 1.6.0 version_**. There are two questions that can guide the improvement program:

1. Are there any differences between 1.5.2 and 1.6.0 users in terms of tutorial participation?
2. Is it possible to roll out 100% game version 1.6.0 to all users, given the available roll-out rate at the moment of this analysis is 50%?

By looking at historical data, we will be able to get a sense of certain patterns that occured in general and in two groups of users, thus understanding the accurate user profiles and coming up with corresponding tactics.

> **<u>Main task</u>**: Analyze historical game data to identify the patterns in game play of users in version 1.5.2 and 1.6.0.

# **Data sources**

The data being used in this analysis is ranged from 28/10/2023 to 10/11/2023. The data is officially collected and provided by INDIEZ and made available [here](https:\drive.google.com\file\d\1aNXHaU1iZIrn5siF6i5EwjrxmAkct47d\view). 

The whole process below, from cleaning to analyzing, is performed on Azure Data Studio using SQL. Due to the disadvantages of performing statistical procedures by SQL, some statistical information of the data will be extracted using other specialized tools.

# **Analysis**

## **Inspecting the data**

The query below gives a sneak peek of the data.

In [1]:
SELECT 
    TOP 10 *
FROM
    indiez..data_interview

(10 rows affected)

Total execution time: 00:00:00.084

column1,date_time,event_name,level,user,day_diff,day0,mode_game,win,reason_to_die,quantity,version
40,2023-10-28,game_start,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,NULL,NULL,0,1.6.0
41,2023-10-28,game_start,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,NULL,NULL,0,1.6.0
42,2023-10-28,game_start,2,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,NULL,NULL,0,1.6.0
43,2023-10-28,game_start,3,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,NULL,NULL,0,1.6.0
44,2023-10-28,game_start,4,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,NULL,NULL,0,1.6.0
45,2023-10-30,game_start,5,1cffd052-4616-4d29-bfd5-950e23b763d4,2,2023-10-28,normal,NULL,NULL,0,1.6.0
46,2023-10-30,game_start,5,1cffd052-4616-4d29-bfd5-950e23b763d4,2,2023-10-28,normal,NULL,NULL,0,1.6.0
47,2023-11-02,game_start,5,1cffd052-4616-4d29-bfd5-950e23b763d4,5,2023-10-28,normal,NULL,NULL,0,1.6.0
48,2023-10-28,game_end,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,0,out_of_lives,31,1.6.0
49,2023-10-28,game_end,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,2023-10-28,normal,1,NULL,18,1.6.0


From the data, we can generally see that:

- One user can have multiple entries based on different `event_name`, day, `mode_game` and win-lose status of the level. After extracting, it reveals that there are **_6671 users playing version 1.5.2_** and **_6929 users playing version 1.6.0_**.
- Mostly, players lost a round because of running out of lives. However, there exists another reason of losing which is '_out of lives plus_'. It can be assumed from the gameplay that '_out of lives plus_' occured when players ran out of 3 given lives, watched ads to gain more life to continue the level but still lost.

## **Tutorial completion rate**

The query below is written to reveal the percentage of players from both versions who had completed the tutorial game.

In [1]:
SELECT
    total_table.version,
    total_users,
    tutorial_players,
    (CAST(tutorial_players AS float)/CAST(total_users AS float))*100 AS percentage
FROM 
    (SELECT
        version,
        COUNT(DISTINCT [user]) AS total_users
    FROM 
        indiez..data_interview
    group by 
        version) AS total_table
    JOIN
    (SELECT
        version,
        COUNT(DISTINCT [user]) AS tutorial_players
    FROM
        indiez..data_interview
    WHERE
        quantity = -2
    GROUP BY
        [version]) AS tutorial_table
    ON total_table.version = tutorial_table.[version]

(2 rows affected)

Total execution time: 00:00:00.917

version,total_users,tutorial_players,percentage
1.5.2,6671,6341,95.05321540998352
1.6.0,6929,6571,94.83330927983836


It can be seen that there were approximately 95% players from version 1.5.2 who completed the in-game tutorial, while in 1.6.0 the number was about 94.8%. There is barely any difference between two versions: Most users would finish the in-game tutorial before getting started with the first levels.

## **Retention rate**

The table below shows the retention rate of players in version 1.5.2 and 1.6.0. Since the available data only records the activities in 7 consecutive days since the players' first time opening the game in the given time range, the retention rate is calculated in this 7-day timeframe only.

In [2]:
SELECT
    table_152.day_diff,
    players_152,
    percentage_152,
    players_160,
    percentage_160
FROM (
    SELECT
        day_diff,
        COUNT(day_diff) AS players_152,
        ROUND(
            (CAST(COUNT(day_diff) AS float)/(
                SELECT 
                    COUNT(DISTINCT[user]) 
                FROM 
                    indiez..data_interview 
                WHERE 
                    version = '1.5.2'
                ))*100, 2) AS percentage_152
    FROM 
        (
        SELECT
            DISTINCT [user],
            day_diff,
            version
        FROM
            indiez..data_interview
        ) AS day_diff_table
    WHERE
        version = '1.5.2'
    GROUP BY
        day_diff) AS table_152

    JOIN (
    SELECT
        day_diff,
        COUNT(day_diff) AS players_160,
        ROUND(
            (CAST(COUNT(day_diff) AS float)/(
                SELECT 
                    COUNT(DISTINCT[user]) 
                FROM    
                    indiez..data_interview 
                WHERE 
                    version = '1.6.0'
            ))*100, 2) AS percentage_160
    FROM 
        (
        SELECT
            DISTINCT [user],
            day_diff,
            version
        FROM
            indiez..data_interview
        ) AS day_diff_table
    WHERE
        version = '1.6.0'
    GROUP BY
        day_diff) AS table_160

    ON table_152.day_diff = table_160.day_diff
WHERE
    table_152.day_diff != 0
ORDER BY
    table_152.day_diff

(7 rows affected)

Total execution time: 00:00:01.644

day_diff,players_152,percentage_152,players_160,percentage_160
1,1780,26.68,1981,28.59
2,891,13.36,1036,14.95
3,517,7.75,714,10.3
4,385,5.77,520,7.5
5,275,4.12,423,6.1
6,202,3.03,334,4.82
7,183,2.74,289,4.17


An acceptable and prominent retention rate for mobile games on day 1 is usually about 30-40%. Both versions had the retention rate of about 27-28%, which was a very good starting point. However, the game was not stable since there was a drop in retention rate of both versions since day 2. Until day 7, the retention rate of version 1.5.2 was 2.74% while that of version 1.6.0 was 4.17%.

When comparing two versions, it can be seen that the retention rate of version 1.6.0 is slightly higher than 1.5.2 throughout the first 7 days, which means there was a higher chance thats players in version 1.6.0 return to the game. This finding can be a support for the hypothesis that version 1.6.0 has higher UX.

## **Average user engagement**

<span style="color: var(--vscode-foreground);">Due to disadvantages in extracting statistical data using SQL,</span> <span style="color: var(--vscode-foreground);">to check the statistical information of user engagement, other specialized statistical tools were used. The result is shown in the following images.</span>

### <span style="color: var(--vscode-foreground);"><b>Average user engagement in version 1.5.2</b></span>

![](.\user-engagement-152.png)

### **Average user engagement in version 1.6.0**

![](.\user-engagement-160.png)

On average, in both versions, users engaged in the game around 5 times in 7 days. However, the highest engagement in version 1.5.2 is much higher than that of version 1.6.0, 1249 times in 7 days compared to 896 times. Beside that, other statistical information of version 1.6.0 is higher comparing to 1.5.2.

## **Maximum level by player**

The two queries below show the maximum levels of all players from version 1.5.2 and 1.6.0.

In [ ]:
SELECT
    DISTINCT [user],
    MAX(level) OVER (PARTITION BY [user]) AS maximum_level
FROM 
    indiez..data_interview
WHERE
    version = '1.5.2'

In [ ]:
SELECT
    DISTINCT [user],
    MAX(level) OVER (PARTITION BY [user]) AS maximum_level
FROM 
    indiez..data_interview
WHERE
    version = '1.6.0'

Once again, other statistical tools were used to summarize the above information about the maximum effort of players from both versions. The result is shown in the following images.

### **Maximum effort by version 1.5.2:**

![](.\statistical-info-152.png)

### **Maximum effort by version 1.6.0:**

![](.\statistical-info-160.png)

On average, in both versions, users would play up to level 6 and stop using the app. In contrast to user engagement, the maximum level players in 1.6.0 could reach in 7 days was level 519, while players in version 1.5.2 could only reach level 376 in the same time range. Similar to the user engagement, version 1.6.0 also witnessed an increase in median, quartiles and average, which shows quite a positive view of the improvement program.

## **1-round players**

When inspecting the data, it can be seen that there are some players who stopped after playing the first level. Hypothetically, this can be used to evaluate the efficacy of the in-game tutorial.

The table below shows the proportion of players who stopped playing the game after the first level.

In [10]:
SELECT
    total_table.version,
    stop_after_lvl_1,
    total_players,
    ROUND((CAST(stop_after_lvl_1 AS float)/CAST(total_players AS float))*100, 2) AS percentage
FROM (
    SELECT
        version,
        COUNT(DISTINCT [user]) AS total_players
    FROM
        indiez..data_interview
    GROUP BY
        version) AS total_table
    JOIN 
    (SELECT
        version,
        COUNT(maximum_level) AS stop_after_lvl_1
    FROM 
        (SELECT
            DISTINCT [user],
            MAX(level) OVER (PARTITION BY [user]) AS maximum_level,
            version
        FROM 
            indiez..data_interview) AS maximum_attempt
    WHERE 
        maximum_level = 1
    GROUP BY
        version) AS temptable
    ON temptable.version = total_table.version

(2 rows affected)

Total execution time: 00:00:03.728

version,stop_after_lvl_1,total_players,percentage
1.5.2,434,6671,6.51
1.6.0,500,6929,7.22


In [11]:
SELECT
    temptable1.version,
    lost_lvl_1,
    stop_after_lvl_1,
    ROUND((CAST(lost_lvl_1 AS float)/CAST(stop_after_lvl_1 AS float))*100, 2) AS percentage
FROM (
    SELECT
        version,
        COUNT(maximum_level) AS lost_lvl_1
    FROM 
        (SELECT
            DISTINCT [user],
            MAX(level) OVER (PARTITION BY [user]) AS maximum_level,
            version,
            win
        FROM 
            indiez..data_interview) AS maximum_attempt
    WHERE 
        maximum_level = 1
        AND
        win = 0
    GROUP BY
        version) AS temptable1
    JOIN 
    (SELECT
        version,
        COUNT(maximum_level) AS stop_after_lvl_1
    FROM 
        (SELECT
            DISTINCT [user],
            MAX(level) OVER (PARTITION BY [user]) AS maximum_level,
            version
        FROM 
            indiez..data_interview) AS maximum_attempt
    WHERE 
        maximum_level = 1
    GROUP BY
        version) AS temptable2
    ON temptable1.version = temptable2.version

(2 rows affected)

Total execution time: 00:00:06.264

version,lost_lvl_1,stop_after_lvl_1,percentage
1.5.2,145,434,33.41
1.6.0,131,500,26.2


In version 1.5.2, about 6.5% players stopped playing the game after level 1, while the number was about 7.2% in version 1.6.0. Among these two groups, 33.4% of version 1.5.2 players stopped playing the game after losing level 1, while the number is 26.2% in version 1.6.0. <span style="color: var(--vscode-foreground);">A hypothesis can arise at this point: Some players stopped playing the game because of being unclear about the game's rules.</span>

However, after looking into the tutorial completion rate of this group of players, it is recognized that none of these players ignored the in-game tutorial, which has turned down the hypothesis.

# **Key findings**

The table below shows the key findings in this analysis with the timeframe of the **_first 7 days after users' first activities in the game_**.

|  | Version 1.5.2 | Version 1.6.0 |
| --- | :-: | :-: |
| Tutorial engagement | 95% | 94.8% |
| Retention rate | 26.7% → 2.7% | 28.8% → 4.2%<br>(slightly higher) |
| Average user engagement (times) | 5 | 5 |
| Average maximum level | 6 | 6 |
| Players stopped playing after level 1<br>(compared to total) | 6.5% | 7.2% |
| Players lost level 1 and stopped playing<br>(compared to total players that stopped playing after level 1) | 33.4% | 26.2% |

# **Conclusion**

## **Tutorial engagement**

- In both versions, a very high percentage of new players participated in the in-game tutorial.
- <span style="color: var(--vscode-foreground);">There is a high percentage of players who stopped playing the game after only the first level, nearly 30% of whom stopped playing after losing. Although it can be inferred that this group of users stopped playing the game because of being unclear about the game's rules, the tutorial completion rate has shown that all of these players finished the tutorial game.</span>
- <span style="color: var(--vscode-foreground);">One possible factor that shows the impacts of tutorial-enhancement to improving UX is the retention rate: More users of version 1.6.0 continued playing the game than that of version 1.5.2.</span>

## **Possibility to reach 100% roll-out rate**

- Although differences in two versions can be seen after analyzing, they are not significant enough to conclude about the prominence of version 1.6.0 to reach 100% roll-out rate.
- The retention rate of version 1.6.0 was promising at the beginning of the game. However, it can be seen that although improvement in version 1.6.0 had resulted in more users coming back for the game, the rate still witnessed a dramatic drop from day 2.
- On average, in both versions, players engaged in the game for about 5 times in 7 days and played up to level 6. Considering the changes in user guidance, this can be inferred that there was not much improvement in the ability to retain players.

# **Recommendation**

From my point of view, based on the findings that were inferred from the data and the personal experience I have had with the game of both versions so far, there are some changes that can be made to the game to enhance the attrativeness and improve UX.

- **_Tutorial gameplay can be clearer:_** More explanations, using either words or illustrations, can be on the screen so that players can be more clearly stated about how the game works. The explanation can be in the form of a dynamic slideshow, showing the possible moves to win the round.
- **_User interface can be improved:_** The elements on the screen can be highlighted and explained at the beginning of the tutorial game.